In [ ]:
import numpy as np
import xarray as xr
import os,errno
import sys
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import salem
import scipy.stats as st
import cartopy.crs as ccrs

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
yrs=np.arange(1940,2024,1)

ds=xr.open_dataset(dir_data+'merra2_gridareas.nc')
areas=ds['cell_area']/1e6

ivt_array_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
ivt_max_array_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_array_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_max_array_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_array_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_max_array_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_array_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_max_array_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]


ivt_array_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
ivt_max_array_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_array_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_max_array_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_array_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_max_array_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_array_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_max_array_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]

ivt_array_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
ivt_max_array_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_array_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_max_array_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_array_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_max_array_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_array_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_max_array_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]


ivt_list_polar=[]
ivt_max_list_polar=[]
iwv_list_polar=[]
iwv_max_list_polar=[]
s_list_polar=[]
s_max_list_polar=[]
mfc_list_polar=[]
mfc_max_list_polar=[]

ivt_list_fixed=[]
ivt_max_list_fixed=[]
iwv_list_fixed=[]
iwv_max_list_fixed=[]
s_list_fixed=[]
s_max_list_fixed=[]
mfc_list_fixed=[]
mfc_max_list_fixed=[]

ivt_list_fixed_poleward=[]
ivt_max_list_fixed_poleward=[]
iwv_list_fixed_poleward=[]
iwv_max_list_fixed_poleward=[]
s_list_fixed_poleward=[]
s_max_list_fixed_poleward=[]
mfc_list_fixed_poleward=[]
mfc_max_list_fixed_poleward=[]


for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    if 1==1:
        ds=xr.open_dataset(dir2+'era5.ar.labels.model.variable.'+str(year)+'.nc')
        labeled=ds['ar_labeled']
        
        if i==0:#land mask option
            ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
            t2m=ds['t2m'][0,:,:]
            land_mask=t2m/t2m
            land_mask=land_mask.fillna(0)

            lon_east=[x for x in land_mask.longitude.values if x>=180.]
            lon_west=[x for x in land_mask.longitude.values if x<180.]
            l_east=land_mask.sel(longitude=lon_east)
            l_west=land_mask.sel(longitude=lon_west)
            l_east['longitude']=[x-360. for x in lon_east]
            land_mask=xr.concat([l_east,l_west],dim='longitude')

            dk=xr.Dataset()
            dk['land']=(('lat','lon'),land_mask.values)
            dk.coords['lat']=land_mask.latitude.values
            dk.coords['lon']=land_mask.longitude.values

            land_mask=dk['land']
            land_mask_interp=land_mask.interp_like(labeled[0,:,:])
        
        ds=xr.open_dataset(dir2+'era_5_ivtinterp_'+str(year)+'.nc')

        ivt=ds['ivt'][:,:,:]
        ds=xr.open_dataset(dir2+'era_5_iwvinterp_'+str(year)+'.nc')
        iwv=ds['iwv'][:,:,:]
        ds=xr.open_dataset(dir2+'era_5_u850interp_'+str(year)+'.nc')
        u=ds['u']
        ds=xr.open_dataset(dir2+'era_5_v850interp_'+str(year)+'.nc')
        v=ds['v']
        s=np.sqrt(u**2+v**2)
        
        ds=xr.open_dataset(dir2+'era_5_mfcinterp_'+str(year)+'.nc')#era5 is divergence
        mfc=ds['mfc'][:,:,:]*-1.
        
        
        ivt['lon']=labeled.lon.values
        iwv['lon']=labeled.lon.values
        s['lon']=labeled.lon.values
        mfc['lon']=labeled.lon.values
        
        ivt['lat']=labeled.lat.values
        iwv['lat']=labeled.lat.values
        s['lat']=labeled.lat.values
        mfc['lat']=labeled.lat.values

        if i==0:
            weights=np.cos(np.deg2rad(ds.lat))
        
        ivt_list_year=[]
        ivt_max_list_year=[]
        iwv_list_year=[]
        iwv_max_list_year=[]
        s_list_year=[]
        s_max_list_year=[]
        mfc_list_year=[]
        mfc_max_list_year=[]
        
        ivt_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        ivt_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        
        
        num_list_year=[]
        nums_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]

        count=0
        for j in range(labeled.time.size):
        #if 1==0:
            #print(j)
            date=pd.to_datetime(labeled.time.values[j])
            month=date.month

            label_sel=labeled[j,:,:].where(land_mask_interp<0.5)
            unique=np.unique(label_sel.values)
            unique=[x for x in unique if x>0]
            num=len(unique)
            
            ivt_sel=ivt[j,:,:]
            iwv_sel=iwv[j,:,:]
            s_sel=s[j,:,:]
            mfc_sel=mfc[j,:,:]

            num_day=0
            for k in range(len(unique)):
                sel_feature=label_sel.where(label_sel==unique[k])
                area_feature=areas.where(sel_feature>0)
                area_feature=area_feature.sum(skipna=True).values.tolist()
                
                if area_feature>0:
                    ivt_sel1=ivt_sel.where(sel_feature>0)
                    ivt_weighted=ivt_sel1.weighted(weights)
                    ivt_mean=ivt_weighted.mean(skipna=True).values.tolist()
                    ivt_max=ivt_sel1.max().values.tolist()

                    iwv_sel1=iwv_sel.where(sel_feature>0)
                    iwv_weighted=iwv_sel1.weighted(weights)
                    iwv_mean=iwv_weighted.mean(skipna=True).values.tolist()
                    iwv_max=iwv_sel1.max().values.tolist()

                    s_sel1=s_sel.where(sel_feature>0)
                    s_weighted=s_sel1.weighted(weights)
                    s_mean=s_weighted.mean(skipna=True).values.tolist()
                    s_max=s_sel1.max().values.tolist()

                    mfc_sel1=mfc_sel.where(sel_feature>0)
                    mfc_weighted=mfc_sel1.weighted(weights)
                    mfc_mean=mfc_weighted.mean(skipna=True).values.tolist()
                    mfc_max=mfc_sel1.max().values.tolist()

                    ivt_list_year.append(ivt_mean)
                    ivt_max_list_year.append(ivt_max)
                    iwv_list_year.append(iwv_mean)
                    iwv_max_list_year.append(iwv_max)
                    s_list_year.append(s_mean)
                    s_max_list_year.append(s_max)
                    mfc_list_year.append(mfc_mean)
                    mfc_max_list_year.append(mfc_max)
                    
                    if 1==0:
                        fig=plt.figure(figsize=(18,12))
                        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
                        ax.coastlines(resolution='10m')
                        ax.pcolormesh(mfc_sel.lon.values,mfc_sel.lat.values,ivt_sel1,vmin=0,vmax=800,cmap=plt.cm.seismic)
                        plt.show()

                        fig=plt.figure(figsize=(18,12))
                        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
                        ax.coastlines(resolution='10m')
                        ax.pcolormesh(mfc_sel.lon.values,mfc_sel.lat.values,mfc_sel1,vmin=-20e-5,vmax=20e-5,cmap=plt.cm.seismic)
                        plt.show()
                        
                        if k==len(unique)-1:
                            sys.exit()

                    ivt_array_year[month-1].append(ivt_mean)
                    ivt_max_array_year[month-1].append(ivt_max)
                    iwv_array_year[month-1].append(iwv_mean)
                    iwv_max_array_year[month-1].append(iwv_max)
                    s_array_year[month-1].append(s_mean)
                    s_max_array_year[month-1].append(s_max)
                    mfc_array_year[month-1].append(mfc_mean)
                    mfc_max_array_year[month-1].append(mfc_max)
                    
                    num_day=num_day+1
                    
                    if 1==0:
                        fig=plt.figure(figsize=(18,12))
                        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
                        ax.coastlines(resolution='10m')
                        ax.pcolormesh(sel_feature.lon.values,sel_feature.lat.values,ivt_sel,
                                     vmin=0,vmax=800,cmap=plt.cm.Greys,alpha=1)
                        #ax.pcolormesh(sel_feature.lon.values,sel_feature.lat.values,ivt_sel1,
                        #             vmin=0,vmax=800,cmap=plt.cm.Greys)

                        print('IVT:'+str(ivt_mean)+','+str(ivt_max))
                        print('IWV:'+str(iwv_mean)+','+str(iwv_max))
                        print('S:'+str(s_mean)+','+str(s_max))
                        #print('IVTDIV:'+str(ivtdiv_mean)+','+str(ivtdiv_max))
                        print('MFC:'+str(mfc_mean)+','+str(mfc_max))
                        plt.show()

            num_list_year.append(num_day)
            nums_array_year[month-1].append(num_day)

        s_list_year=[x for x in s_list_year if x>=0]
        s_max_list_year=[x for x in s_max_list_year if x>=0]
        
        mean_ivt=float(sum(ivt_list_year))/float(len(ivt_list_year))
        max_ivt=float(sum(ivt_max_list_year))/float(len(ivt_max_list_year))
        mean_iwv=float(sum(iwv_list_year))/float(len(iwv_list_year))
        max_iwv=float(sum(iwv_max_list_year))/float(len(iwv_max_list_year))
        mean_s=float(sum(s_list_year))/float(len(s_list_year))
        max_s=float(sum(s_max_list_year))/float(len(s_max_list_year))
        mean_mfc=float(sum(mfc_list_year))/float(len(mfc_list_year))
        max_mfc=float(sum(mfc_max_list_year))/float(len(mfc_max_list_year))
        
        ivt_list_polar.append(mean_ivt)
        ivt_max_list_polar.append(max_ivt)
        iwv_list_polar.append(mean_iwv)
        iwv_max_list_polar.append(max_iwv)
        s_list_polar.append(mean_s)
        s_max_list_polar.append(max_s)
        mfc_list_polar.append(mean_mfc)
        mfc_max_list_polar.append(max_mfc)
        
        
        for j in range(len(nums_array_year)):
            ivt_array_polar[j].append(float(sum(ivt_array_year[j]))/float(len(ivt_array_year[j])))
            ivt_max_array_polar[j].append(float(sum(ivt_max_array_year[j]))/float(len(ivt_max_array_year[j])))
        
            iwv_array_polar[j].append(float(sum(iwv_array_year[j]))/float(len(iwv_array_year[j])))
            iwv_max_array_polar[j].append(float(sum(iwv_max_array_year[j]))/float(len(iwv_max_array_year[j])))
        
            s_array_polar[j].append(float(sum(s_array_year[j]))/float(len(s_array_year[j])))
            s_max_array_polar[j].append(float(sum(s_max_array_year[j]))/float(len(s_max_array_year[j])))

            mfc_array_polar[j].append(float(sum(mfc_array_year[j]))/float(len(mfc_array_year[j])))
            mfc_max_array_polar[j].append(float(sum(mfc_max_array_year[j]))/float(len(mfc_max_array_year[j])))

        print('A')
        #################

        ds=xr.open_dataset(dir2+'era5.ar.labels.model.'+str(year)+'.nc')
        labeled=ds['ar_labeled'][:,:,:]
        labeled['lon']=ivt.lon.values
        labeled['lat']=ivt.lat.values

        ivt_list_year=[]
        ivt_max_list_year=[]
        iwv_list_year=[]
        iwv_max_list_year=[]
        s_list_year=[]
        s_max_list_year=[]
        mfc_list_year=[]
        mfc_max_list_year=[]
        
        ivt_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        ivt_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        
        num_list_year=[]
        nums_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]

        count=0
        for j in range(labeled.time.size):
            #print(j)
            date=pd.to_datetime(labeled.time.values[j])
            month=date.month

            label_sel=labeled[j,:,:].where(land_mask_interp<0.5)
            unique=np.unique(label_sel.values)
            unique=[x for x in unique if x>0]
            num=len(unique)
            
            ivt_sel=ivt[j,:,:]
            iwv_sel=iwv[j,:,:]
            s_sel=s[j,:,:]
            mfc_sel=mfc[j,:,:]

            num_day=0
            for k in range(len(unique)):
                sel_feature=label_sel.where(label_sel==unique[k])
                area_feature=areas.where(sel_feature>0)
                area_feature=area_feature.sum(skipna=True).values.tolist()

                if area_feature>0:
                    ivt_sel1=ivt_sel.where(sel_feature>0)
                    ivt_weighted=ivt_sel1.weighted(weights)
                    ivt_mean=ivt_weighted.mean(skipna=True).values.tolist()
                    ivt_max=ivt_sel1.max().values.tolist()

                    iwv_sel1=iwv_sel.where(sel_feature>0)
                    iwv_weighted=iwv_sel1.weighted(weights)
                    iwv_mean=iwv_weighted.mean(skipna=True).values.tolist()
                    iwv_max=iwv_sel1.max().values.tolist()

                    s_sel1=s_sel.where(sel_feature>0)
                    s_weighted=s_sel1.weighted(weights)
                    s_mean=s_weighted.mean(skipna=True).values.tolist()
                    s_max=s_sel1.max().values.tolist()

                    mfc_sel1=mfc_sel.where(sel_feature>0)
                    mfc_weighted=mfc_sel1.weighted(weights)
                    mfc_mean=mfc_weighted.mean(skipna=True).values.tolist()
                    mfc_max=mfc_sel1.max().values.tolist()

                    ivt_list_year.append(ivt_mean)
                    ivt_max_list_year.append(ivt_max)
                    iwv_list_year.append(iwv_mean)
                    iwv_max_list_year.append(iwv_max)
                    s_list_year.append(s_mean)
                    s_max_list_year.append(s_max)
                    mfc_list_year.append(mfc_mean)
                    mfc_max_list_year.append(mfc_max)
                    
                    
                    ivt_array_year[month-1].append(ivt_mean)
                    ivt_max_array_year[month-1].append(ivt_max)
                    iwv_array_year[month-1].append(iwv_mean)
                    iwv_max_array_year[month-1].append(iwv_max)
                    s_array_year[month-1].append(s_mean)
                    s_max_array_year[month-1].append(s_max)
                    mfc_array_year[month-1].append(mfc_mean)
                    mfc_max_array_year[month-1].append(mfc_max)
                        
                    num_day=num_day+1

            num_list_year.append(num_day)
            nums_array_year[month-1].append(num_day)

        s_list_year=[x for x in s_list_year if x>=0]
        s_max_list_year=[x for x in s_max_list_year if x>=0]
        
        mean_ivt=float(sum(ivt_list_year))/float(len(ivt_list_year))
        max_ivt=float(sum(ivt_max_list_year))/float(len(ivt_max_list_year))
        mean_iwv=float(sum(iwv_list_year))/float(len(iwv_list_year))
        max_iwv=float(sum(iwv_max_list_year))/float(len(iwv_max_list_year))
        mean_s=float(sum(s_list_year))/float(len(s_list_year))
        max_s=float(sum(s_max_list_year))/float(len(s_max_list_year))
        mean_mfc=float(sum(mfc_list_year))/float(len(mfc_list_year))
        max_mfc=float(sum(mfc_max_list_year))/float(len(mfc_max_list_year))
        
        ivt_list_fixed.append(mean_ivt)
        ivt_max_list_fixed.append(max_ivt)
        iwv_list_fixed.append(mean_iwv)
        iwv_max_list_fixed.append(max_iwv)
        s_list_fixed.append(mean_s)
        s_max_list_fixed.append(max_s)
        mfc_list_fixed.append(mean_mfc)
        mfc_max_list_fixed.append(max_mfc)
        
        for j in range(len(nums_array_year)):
            ivt_array_fixed[j].append(float(sum(ivt_array_year[j]))/float(len(ivt_array_year[j])))
            ivt_max_array_fixed[j].append(float(sum(ivt_max_array_year[j]))/float(len(ivt_max_array_year[j])))
        
            iwv_array_fixed[j].append(float(sum(iwv_array_year[j]))/float(len(iwv_array_year[j])))
            iwv_max_array_fixed[j].append(float(sum(iwv_max_array_year[j]))/float(len(iwv_max_array_year[j])))
        
            s_array_fixed[j].append(float(sum(s_array_year[j]))/float(len(s_array_year[j])))
            s_max_array_fixed[j].append(float(sum(s_max_array_year[j]))/float(len(s_max_array_year[j])))

            mfc_array_fixed[j].append(float(sum(mfc_array_year[j]))/float(len(mfc_array_year[j])))
            mfc_max_array_fixed[j].append(float(sum(mfc_max_array_year[j]))/float(len(mfc_max_array_year[j])))

        print('B')
        
        ds=xr.open_dataset(dir2+'era5.ar.labels.model.poleward.'+str(year)+'.nc')
        labeled=ds['ar_labeled']
        labeled['lon']=ivt.lon.values
        labeled['lat']=ivt.lat.values

        ivt_list_year=[]
        ivt_max_list_year=[]
        iwv_list_year=[]
        iwv_max_list_year=[]
        s_list_year=[]
        s_max_list_year=[]
        mfc_list_year=[]
        mfc_max_list_year=[]
        
        ivt_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        ivt_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        
        num_list_year=[]
        nums_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]

        count=0
        for j in range(labeled.time.size):
            #print(j)
            date=pd.to_datetime(labeled.time.values[j])
            month=date.month

            label_sel=labeled[j,:,:].where(land_mask_interp<0.5)
            unique=np.unique(label_sel.values)
            unique=[x for x in unique if x>0]
            num=len(unique)
            
            ivt_sel=ivt[j,:,:]
            iwv_sel=iwv[j,:,:]
            s_sel=s[j,:,:]
            mfc_sel=mfc[j,:,:]

            num_day=0
            for k in range(len(unique)):
                sel_feature=label_sel.where(label_sel==unique[k])
                area_feature=areas.where(sel_feature>0)
                area_feature=area_feature.sum(skipna=True).values.tolist()

                if area_feature>0:
                    
                    ivt_sel1=ivt_sel.where(sel_feature>0)
                    ivt_weighted=ivt_sel1.weighted(weights)
                    ivt_mean=ivt_weighted.mean(skipna=True).values.tolist()
                    ivt_max=ivt_sel1.max().values.tolist()

                    iwv_sel1=iwv_sel.where(sel_feature>0)
                    iwv_weighted=iwv_sel1.weighted(weights)
                    iwv_mean=iwv_weighted.mean(skipna=True).values.tolist()
                    iwv_max=iwv_sel1.max().values.tolist()

                    s_sel1=s_sel.where(sel_feature>0)
                    s_weighted=s_sel1.weighted(weights)
                    s_mean=s_weighted.mean(skipna=True).values.tolist()
                    s_max=s_sel1.max().values.tolist()
                    
                    mfc_sel1=mfc_sel.where(sel_feature>0)
                    mfc_weighted=mfc_sel1.weighted(weights)
                    mfc_mean=mfc_weighted.mean(skipna=True).values.tolist()
                    mfc_max=mfc_sel1.max().values.tolist()

                    ivt_list_year.append(ivt_mean)
                    ivt_max_list_year.append(ivt_max)
                    iwv_list_year.append(iwv_mean)
                    iwv_max_list_year.append(iwv_max)
                    s_list_year.append(s_mean)
                    s_max_list_year.append(s_max)
                    mfc_list_year.append(mfc_mean)
                    mfc_max_list_year.append(mfc_max)
                    
                    ivt_array_year[month-1].append(ivt_mean)
                    ivt_max_array_year[month-1].append(ivt_max)
                    iwv_array_year[month-1].append(iwv_mean)
                    iwv_max_array_year[month-1].append(iwv_max)
                    s_array_year[month-1].append(s_mean)
                    s_max_array_year[month-1].append(s_max)
                    mfc_array_year[month-1].append(mfc_mean)
                    mfc_max_array_year[month-1].append(mfc_max)
                        
                    num_day=num_day+1

            num_list_year.append(num_day)
            nums_array_year[month-1].append(num_day)

        s_list_year=[x for x in s_list_year if x>=0]
        s_max_list_year=[x for x in s_max_list_year if x>=0]
        
        mean_ivt=float(sum(ivt_list_year))/float(len(ivt_list_year))
        max_ivt=float(sum(ivt_max_list_year))/float(len(ivt_max_list_year))
        mean_iwv=float(sum(iwv_list_year))/float(len(iwv_list_year))
        max_iwv=float(sum(iwv_max_list_year))/float(len(iwv_max_list_year))
        mean_s=float(sum(s_list_year))/float(len(s_list_year))
        max_s=float(sum(s_max_list_year))/float(len(s_max_list_year))
        mean_mfc=float(sum(mfc_list_year))/float(len(mfc_list_year))
        max_mfc=float(sum(mfc_max_list_year))/float(len(mfc_max_list_year))
        
        ivt_list_fixed_poleward.append(mean_ivt)
        ivt_max_list_fixed_poleward.append(max_ivt)
        iwv_list_fixed_poleward.append(mean_iwv)
        iwv_max_list_fixed_poleward.append(max_iwv)
        s_list_fixed_poleward.append(mean_s)
        s_max_list_fixed_poleward.append(max_s)
        mfc_list_fixed_poleward.append(mean_mfc)
        mfc_max_list_fixed_poleward.append(max_mfc)
        
        for j in range(len(nums_array_year)):
            ivt_array_fixed_poleward[j].append(float(sum(ivt_array_year[j]))/float(len(ivt_array_year[j])))
            ivt_max_array_fixed_poleward[j].append(float(sum(ivt_max_array_year[j]))/float(len(ivt_max_array_year[j])))
        
            iwv_array_fixed_poleward[j].append(float(sum(iwv_array_year[j]))/float(len(iwv_array_year[j])))
            iwv_max_array_fixed_poleward[j].append(float(sum(iwv_max_array_year[j]))/float(len(iwv_max_array_year[j])))
        
            s_array_fixed_poleward[j].append(float(sum(s_array_year[j]))/float(len(s_array_year[j])))
            s_max_array_fixed_poleward[j].append(float(sum(s_max_array_year[j]))/float(len(s_max_array_year[j])))

            mfc_array_fixed_poleward[j].append(float(sum(mfc_array_year[j]))/float(len(mfc_array_year[j])))
            mfc_max_array_fixed_poleward[j].append(float(sum(mfc_max_array_year[j]))/float(len(mfc_max_array_year[j])))          
       
    print('C')

In [ ]:
dk=xr.Dataset()

dk['ivt_array_era5_polar']=(('months','years'),np.asarray(ivt_array_polar))
dk['ivt_max_array_era5_polar']=(('months','years'),np.asarray(ivt_max_array_polar))
dk['iwv_array_era5_polar']=(('months','years'),np.asarray(iwv_array_polar))
dk['iwv_max_array_era5_polar']=(('months','years'),np.asarray(iwv_max_array_polar))
dk['s_array_era5_polar']=(('months','years'),np.asarray(s_array_polar))
dk['s_max_array_era5_polar']=(('months','years'),np.asarray(s_max_array_polar))
dk['mfc_array_era5_polar']=(('months','years'),np.asarray(mfc_array_polar))
dk['mfc_max_array_era5_polar']=(('months','years'),np.asarray(mfc_max_array_polar))

dk['ivt_array_era5_fixed']=(('months','years'),np.asarray(ivt_array_fixed))
dk['ivt_max_array_era5_fixed']=(('months','years'),np.asarray(ivt_max_array_fixed))
dk['iwv_array_era5_fixed']=(('months','years'),np.asarray(iwv_array_fixed))
dk['iwv_max_array_era5_fixed']=(('months','years'),np.asarray(iwv_max_array_fixed))
dk['s_array_era5_fixed']=(('months','years'),np.asarray(s_array_fixed))
dk['s_max_array_era5_fixed']=(('months','years'),np.asarray(s_max_array_fixed))
dk['mfc_array_era5_fixed']=(('months','years'),np.asarray(mfc_array_fixed))
dk['mfc_max_array_era5_fixed']=(('months','years'),np.asarray(mfc_max_array_fixed))

dk['ivt_array_era5_fixed_poleward']=(('months','years'),np.asarray(ivt_array_fixed_poleward))
dk['ivt_max_array_era5_fixed_poleward']=(('months','years'),np.asarray(ivt_max_array_fixed_poleward))
dk['iwv_array_era5_fixed_poleward']=(('months','years'),np.asarray(iwv_array_fixed_poleward))
dk['iwv_max_array_era5_fixed_poleward']=(('months','years'),np.asarray(iwv_max_array_fixed_poleward))
dk['s_array_era5_fixed_poleward']=(('months','years'),np.asarray(s_array_fixed_poleward))
dk['s_max_array_era5_fixed_poleward']=(('months','years'),np.asarray(s_max_array_fixed_poleward))
dk['mfc_array_era5_fixed_poleward']=(('months','years'),np.asarray(mfc_array_fixed_poleward))
dk['mfc_max_array_era5_fixed_poleward']=(('months','years'),np.asarray(mfc_max_array_fixed_poleward))

dk['ivt_list_era5_polar']=(('years'),ivt_list_polar)
dk['ivt_max_list_era5_polar']=(('years'),ivt_max_list_polar)
dk['iwv_list_era5_polar']=(('years'),iwv_list_polar)
dk['iwv_max_list_era5_polar']=(('years'),iwv_max_list_polar)
dk['s_list_era5_polar']=(('years'),s_list_polar)
dk['s_max_list_era5_polar']=(('years'),s_max_list_polar)
dk['mfc_list_era5_polar']=(('years'),mfc_list_polar)
dk['mfc_max_list_era5_polar']=(('years'),mfc_max_list_polar)

dk['ivt_list_era5_fixed']=(('years'),ivt_list_fixed)
dk['ivt_max_list_era5_fixed']=(('years'),ivt_max_list_fixed)
dk['iwv_list_era5_fixed']=(('years'),iwv_list_fixed)
dk['iwv_max_list_era5_fixed']=(('years'),iwv_max_list_fixed)
dk['s_list_era5_fixed']=(('years'),s_list_fixed)
dk['s_max_list_era5_fixed']=(('years'),s_max_list_fixed)
dk['mfc_list_era5_fixed']=(('years'),mfc_list_fixed)
dk['mfc_max_list_era5_fixed']=(('years'),mfc_max_list_fixed)

dk['ivt_list_era5_fixed_poleward']=(('years'),ivt_list_fixed_poleward)
dk['ivt_max_list_era5_fixed_poleward']=(('years'),ivt_max_list_fixed_poleward)
dk['iwv_list_era5_fixed_poleward']=(('years'),iwv_list_fixed_poleward)
dk['iwv_max_list_era5_fixed_poleward']=(('years'),iwv_max_list_fixed_poleward)
dk['s_list_era5_fixed_poleward']=(('years'),s_list_fixed_poleward)
dk['s_max_list_era5_fixed_poleward']=(('years'),s_max_list_fixed_poleward)
dk['mfc_list_era5_fixed_poleward']=(('years'),mfc_list_fixed_poleward)
dk['mfc_max_list_era5_fixed_poleward']=(('years'),mfc_max_list_fixed_poleward)

dk.coords['years']=np.arange(1940,2024,1)
dk.coords['months']=np.arange(1,13,1)
try:
    os.remove(dir_data+'ar_chars_synoptic_era5_OCEAN.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'ar_chars_synoptic_era5_OCEAN.nc',mode='w',format='NETCDF4')